In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer

##Preprocesamiento

# Leer la base de datos
url = "https://raw.githubusercontent.com/jclavijo9612/proyectos/main/bank-additional-full.csv"
data = pd.read_csv(url, sep=";")


# Definir el porcentaje de registros a eliminar
percent_to_drop = 0.9

# Calcular el número de registros a eliminar
num_records = len(data)
num_records_to_drop = int(num_records * percent_to_drop)

# Eliminar aleatoriamente el 90% de los registros
data_sampled = data.sample(n=num_records_to_drop, random_state=42)

# Mantener el 10% restante de los registros
data_remaining = data.drop(data_sampled.index)

# Verificar el tamaño del dataset resultante
print("Tamaño del dataset original:", num_records)
print("Tamaño del dataset remanente:", len(data_remaining))

data=data_remaining

# Visualización de los primeros registros
print(data.head())

# Resumen estadístico
print(data.describe())

# Variables categóricas
categorical_cols = ['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Iterar sobre las columnas categóricas y mostrar los valores únicos
for column in categorical_cols:
    unique_values = data[column].unique()
    print(f"Valores únicos en la columna {column}:")
    print(unique_values)
    print()

Tamaño del dataset original: 41188
Tamaño del dataset remanente: 4119
    age           job  marital            education  default housing loan  \
9    25      services   single          high.school       no     yes   no   
55   55    technician  married  professional.course  unknown     yes   no   
60   47  entrepreneur  married    university.degree  unknown      no   no   
77   33      services  married          high.school  unknown     yes   no   
82   38        admin.  married          high.school  unknown      no   no   

      contact month day_of_week  ...  campaign  pdays  previous     poutcome  \
9   telephone   may         mon  ...         1    999         0  nonexistent   
55  telephone   may         mon  ...         1    999         0  nonexistent   
60  telephone   may         mon  ...         1    999         0  nonexistent   
77  telephone   may         mon  ...         2    999         0  nonexistent   
82  telephone   may         mon  ...         2    999         0  no

In [2]:
#Reemplazamos y para que sea 0 y 1
data['y'] = data['y'].replace({'no': 0, 'yes': 1})


#Quitar columna duration
data = data.drop(['duration','day_of_week','default','pdays','campaign'], axis=1 )




In [3]:
## Label Encoding de las variables categorica

# Crear una instancia de LabelEncoder
label_encoder = LabelEncoder()

# Definir el orden para la columna 'month'
month_order = ['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Definir el orden para la columna 'day_of_week'
day_of_week_order = ['mon', 'tue', 'wed', 'thu', 'fri']

for col in categorical_cols:
    if col == 'month':
        label_encoder = LabelEncoder()
        label_encoder.fit(month_order)
        data[col] = label_encoder.transform(data[col])
    else:
        data[col] = label_encoder.fit_transform(data[col])

# Mostrar los primeros registros del conjunto de datos con las variables categóricas codificadas
print(data.head())


    age  job  marital  education  housing  loan  contact  month  previous  \
9    25    7        2          3        2     0        1      6         0   
55   55    9        1          5        2     0        1      6         0   
60   47    2        1          6        0     0        1      6         0   
77   33    7        1          3        2     0        1      6         0   
82   38    0        1          3        0     0        1      6         0   

    poutcome  emp.var.rate  cons.price.idx  cons.conf.idx  euribor3m  \
9          1           1.1          93.994          -36.4      4.857   
55         1           1.1          93.994          -36.4      4.857   
60         1           1.1          93.994          -36.4      4.857   
77         1           1.1          93.994          -36.4      4.857   
82         1           1.1          93.994          -36.4      4.857   

    nr.employed  y  
9        5191.0  0  
55       5191.0  0  
60       5191.0  0  
77       5191.0  0  

In [4]:
##Imputación de nulls

# Crea una instancia de SimpleImputer con la estrategia de imputación deseada (por ejemplo, media)
imputer = SimpleImputer(strategy='mean')

# Imputa los valores nulos en el dataset
data_imputed = imputer.fit_transform(data)

# Convierte el resultado nuevamente en un DataFrame
data_imputed = pd.DataFrame(data_imputed, columns=data.columns)

data = data_imputed


In [5]:
##Normalización de variables numéricas

# Crea una instancia del escalador MinMaxScaler
scaler = MinMaxScaler()

# Selecciona las columnas numéricas a normalizar
numeric_columns = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

# Aplica la normalización min-max a las columnas numéricas
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

X = data.drop('y', axis=1)
y = data['y']

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en el GridSearch
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

# Crear el modelo de regresión logística
model = LogisticRegression(solver='liblinear')
# Realizar GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = LogisticRegression(solver='liblinear', **best_params)

# Realizar validación cruzada y calcular la puntuación promedio
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
mean_cv_score = cv_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error (precisión)
accuracy = accuracy_score(y_test, y_pred)

# Imprimir los resultados
print("Resultados del modelo de regresión lineal:")
print("Mejores hiperparámetros encontrados:", best_params)
print("Puntuación promedio de validación cruzada:", mean_cv_score)
print("Precisión en el conjunto de prueba:", accuracy)



In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en el GridSearch
param_grid = {
    'penalty': ['l1'],
    'C': [ 1]
}

# Crear el modelo de regresión logística
model = LogisticRegression(solver='liblinear')
# Realizar GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = LogisticRegression(solver='liblinear', **best_params)

# Realizar validación cruzada y calcular la puntuación promedio
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
mean_cv_score = cv_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error (precisión)
accuracy = accuracy_score(y_test, y_pred)

# Imprimir los resultados
print("Resultados del modelo de regresión lineal:")
print("Mejores hiperparámetros encontrados:", best_params)
print("Puntuación promedio de validación cruzada:", mean_cv_score)
print("Precisión en el conjunto de prueba:", accuracy)



In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearch
parameters = {
    'criterion': ['gini'],
    'max_depth': [10],
    'min_samples_split': [ 10],
    'min_samples_leaf': [1],
    'random_state': [42]
}

# Crear un modelo de árbol de decisión con validación cruzada
grid_search = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = DecisionTreeClassifier(**best_params)

# Realizar validación cruzada y calcular el puntaje promedio
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
avg_score = cross_val_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de pruebas
y_pred = best_model.predict(X_test)

# Calcular el error y mostrar los resultados
accuracy = accuracy_score(y_test, y_pred)
print("Mejores hiperparámetros encontrados:", best_params)
print("Puntaje promedio de validación cruzada:", avg_score)
print("Exactitud en el conjunto de pruebas:", accuracy)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score


smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearch
parameters = {
    'n_estimators': [10, 50, 100],
    'max_samples': [0.5, 0.8, 1.0],
    'max_features': [0.5, 0.8, 1.0],
    'random_state': [42]
}

# Crear un modelo de Bagging con validación cruzada
grid_search = GridSearchCV(BaggingClassifier(), parameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = BaggingClassifier(**best_params)

# Realizar validación cruzada y calcular el puntaje promedio
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
avg_score = cross_val_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de pruebas
y_pred = best_model.predict(X_test)

# Calcular el error y mostrar los resultados
accuracy = accuracy_score(y_test, y_pred)
print("Mejores hiperparámetros encontrados:", best_params)
print("Puntaje promedio de validación cruzada:", avg_score)
print("Exactitud en el conjunto de pruebas:", accuracy)


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearch
parameters = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [42]
}

# Crear un modelo de RandomForest con validación cruzada
grid_search = GridSearchCV(RandomForestClassifier(), parameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = RandomForestClassifier(**best_params)

# Realizar validación cruzada y calcular el puntaje promedio
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
avg_score = cross_val_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de pruebas
y_pred = best_model.predict(X_test)

# Calcular el error y mostrar los resultados
accuracy = accuracy_score(y_test, y_pred)
print("Mejores hiperparámetros encontrados:", best_params)
print("Puntaje promedio de validación cruzada:", avg_score)
print("Exactitud en el conjunto de pruebas:", accuracy)


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearch
parameters = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5, 10],
    'subsample': [0.8, 1.0],
    'random_state': [42]
}

# Crear un modelo de GradientBoosting con validación cruzada
grid_search = GridSearchCV(GradientBoostingClassifier(), parameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = GradientBoostingClassifier(**best_params)

# Realizar validación cruzada y calcular el puntaje promedio
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
avg_score = cross_val_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de pruebas
y_pred = best_model.predict(X_test)

# Calcular el error y mostrar los resultados
accuracy = accuracy_score(y_test, y_pred)
print("Mejores hiperparámetros encontrados:", best_params)
print("Puntaje promedio de validación cruzada:", avg_score)
print("Exactitud en el conjunto de pruebas:", accuracy)


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)
# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearch
parameters = {
    'n_estimators': [100],
    'learning_rate': [0.1],
    'max_depth': [3],
    'subsample': [0.8],
    'random_state': [42]
}

# Crear un modelo de GradientBoosting con validación cruzada
grid_search = GridSearchCV(GradientBoostingClassifier(), parameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = GradientBoostingClassifier(**best_params)

# Realizar validación cruzada y calcular el puntaje promedio
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
avg_score = cross_val_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de pruebas
y_pred = best_model.predict(X_test)

# Calcular el error y mostrar los resultados
accuracy = accuracy_score(y_test, y_pred)
print("Mejores hiperparámetros encontrados:", best_params)
print("Puntaje promedio de validación cruzada:", avg_score)
print("Exactitud en el conjunto de pruebas:", accuracy)

In [ ]:
pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearchCV
params = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [100],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8]
}

# Crear un modelo de XGBoost
xgb_model = xgb.XGBClassifier()

# Realizar GridSearchCV
grid_search = GridSearchCV(xgb_model, params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = xgb.XGBClassifier(**best_params)

# Realizar validación cruzada y calcular el puntaje promedio
scores = cross_val_score(best_model, X_train, y_train, cv=5)
mean_score = np.mean(scores)

# Ajustar el modelo final con todos los datos
best_model.fit(X, y)

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error (por ejemplo, precisión)
error = 1 - accuracy_score(y_test, y_pred)

# Imprimir los resultados, los mejores hiperparámetros y el puntaje obtenido
print("Resultados del modelo XGBoost:")
print("Mejores hiperparámetros:", best_params)
print("Puntaje promedio de validación cruzada:", mean_score)
print("Error en el conjunto de prueba:", error)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearchCV
params = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9]
}

# Crear un modelo de XGBoost
xgb_model = xgb.XGBClassifier()

# Realizar GridSearchCV
grid_search = GridSearchCV(xgb_model, params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_model = xgb.XGBClassifier(**best_params)

# Realizar validación cruzada y calcular el puntaje promedio
scores = cross_val_score(best_model, X_train, y_train, cv=5)
mean_score = np.mean(scores)

# Ajustar el modelo final con todos los datos
best_model.fit(X, y)

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error (por ejemplo, precisión)
error = 1 - accuracy_score(y_test, y_pred)

# Imprimir los resultados, los mejores hiperparámetros y el puntaje obtenido
print("Resultados del modelo XGBoost:")
print("Mejores hiperparámetros:", best_params)
print("Puntaje promedio de validación cruzada:", mean_score)
print("Error en el conjunto de prueba:", error)


In [ ]:
print(data)

In [ ]:
data.drop(['campaign', 'previous'], axis=1, inplace=True)


In [ ]:
print(data)

In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

X = data.drop('y', axis=1)
y = data['y']

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearchCV
parameters = {
    'hidden_layer_sizes': [(100)],
    'activation': ['tanh'],
    'solver': ['sgd'],
    'alpha': [0.0001],
}

# Crear el modelo de Multi-Layer Perceptron
mlp = MLPClassifier(random_state=42)

# Realizar GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(mlp, parameters, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_mlp = MLPClassifier(**best_params, random_state=42)

# Realizar validación cruzada para obtener la puntuación promedio
cv_scores = cross_val_score(best_mlp, X_train, y_train, cv=5)
mean_cv_score = cv_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_mlp.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_mlp.predict(X_test)

# Calcular la precisión del modelo final en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)

# Imprimir los resultados
print("Mejores hiperparámetros:", best_params)
print("Puntuación promedio de validación cruzada:", mean_cv_score)
print("Precisión en el conjunto de prueba:", accuracy)


C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural

Mejores hiperparámetros: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': 100, 'solver': 'sgd'}
Puntuación promedio de validación cruzada: 0.8813353566009103
Precisión en el conjunto de prueba: 0.9065533980582524


C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en GridSearchCV
parameters = {
    'hidden_layer_sizes': [(100)],
    'activation': [ 'tanh'],
    'solver': ['sgd'],
    'alpha': [0.0001],
}

# Crear el modelo de Multi-Layer Perceptron
mlp = MLPClassifier(random_state=42)

# Realizar GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(mlp, parameters, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_mlp = MLPClassifier(**best_params, random_state=42)

# Realizar validación cruzada para obtener la puntuación promedio
cv_scores = cross_val_score(best_mlp, X_train, y_train, cv=5)
mean_cv_score = cv_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_mlp.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_mlp.predict(X_test)

# Calcular la precisión del modelo final en el conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)

# Imprimir los resultados
print("Mejores hiperparámetros:", best_params)
print("Puntuación promedio de validación cruzada:", mean_cv_score)
print("Precisión en el conjunto de prueba:", accuracy)


C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural

Mejores hiperparámetros: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': 100, 'solver': 'sgd'}
Puntuación promedio de validación cruzada: 0.7319507849411848
Precisión en el conjunto de prueba: 0.7235213204951857


C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir los hiperparámetros a ajustar en el GridSearch
param_grid = {
    'C': [0.1],
    'kernel': ['linear'],
    'gamma': ['scale']
}

# Crear el modelo SVC
svc = SVC()

# Realizar la búsqueda de hiperparámetros mediante validación cruzada
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros encontrados
best_params = grid_search.best_params_

# Crear un nuevo modelo con los mejores hiperparámetros
best_svc = SVC(**best_params)

# Realizar validación cruzada y obtener el puntaje promedio
cv_scores = cross_val_score(best_svc, X_train, y_train, cv=5)
cv_score_mean = cv_scores.mean()

# Ajustar el modelo final con todos los datos
best_svc.fit(X, y)

# Realizar predicciones en el conjunto de test
y_pred = best_svc.predict(X_test)

# Calcular el error
error = 1 - accuracy_score(y_test, y_pred)

# Imprimir los resultados
print("Mejores hiperparámetros:", best_params)
print("Puntaje de validación cruzada:", cv_score_mean)
print("Error en el conjunto de test:", error)


Mejores hiperparámetros: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Puntaje de validación cruzada: 0.7190507426876221
Error en el conjunto de test: 0.39958734525447037


In [13]:
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

X = data.drop('y', axis=1)
y = data['y']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Definir el clasificador base (Gradient Boosting)
base_classifier = GradientBoostingClassifier()

# Definir los parámetros a ajustar en el gridsearch
param_grid = {
    'n_estimators': [10],  # Número de estimadores en el ensamblado
    'max_samples': [0.5],  # Tamaño de la muestra para cada estimador
    'max_features': [0.5, 0.8],  # Número de características para cada estimador
}

# Crear el modelo de Bagging con el clasificador base
bagging_model = BaggingClassifier(base_estimator=base_classifier)

# Realizar la búsqueda de hiperparámetros mediante validación cruzada
grid_search = GridSearchCV(bagging_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y crear un nuevo modelo con ellos
best_params = grid_search.best_params_
best_model = BaggingClassifier(base_estimator=base_classifier, **best_params)

# Realizar validación cruzada para obtener el puntaje promedio
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
average_score = cv_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error (por ejemplo, precisión o exactitud)
error = 1 - accuracy_score(y_test, y_pred)

# Imprimir los resultados
print("Mejores hiperparámetros:", best_params)
print("Puntaje promedio de validación cruzada:", average_score)
print("Error en el conjunto de prueba:", error)


Mejores hiperparámetros: {'max_features': 0.8, 'max_samples': 0.5, 'n_estimators': 10}
Puntaje promedio de validación cruzada: 0.9203902988183776
Error en el conjunto de prueba: 0.08459422283356255


In [15]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

X = data.drop('y', axis=1)
y = data['y']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los clasificadores base
logistic_regression = LogisticRegression()
random_forest = RandomForestClassifier()
bagging = BaggingClassifier()
gradient_boosting = GradientBoostingClassifier()
xgboost = XGBClassifier()
svc = SVC()

# Definir el ensamblador de Stacking
estimators = [
    ('logistic_regression', logistic_regression),
    ('random_forest', random_forest),
    ('bagging', bagging),
    ('gradient_boosting', gradient_boosting),
    ('xgboost', xgboost),
    ('svc', svc)
]
stacking_model = StackingClassifier(estimators=estimators)

# Definir los parámetros a ajustar en el gridsearch
param_grid = {
    'logistic_regression__C': [1],
    'random_forest__n_estimators': [200],
    'gradient_boosting__n_estimators': [100],
    'xgboost__n_estimators': [100],
    'svc__C': [0,1]
}

# Realizar la búsqueda de hiperparámetros mediante validación cruzada
grid_search = GridSearchCV(stacking_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros y crear un nuevo modelo con ellos
best_params = grid_search.best_params_
best_model = StackingClassifier(estimators=estimators, **best_params)

# Realizar validación cruzada para obtener el puntaje promedio
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
average_score = cv_scores.mean()

# Ajustar el modelo final con todos los datos de entrenamiento
best_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular el error (por ejemplo, precisión o exactitud)
error = 1 - accuracy_score(y_test, y_pred)

# Imprimir los resultados
print("Mejores hiperparámetros:", best_params)
print("Puntaje promedio de validación cruzada:", average_score)
print("Error en el conjunto de prueba:", error)


C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Clju2001\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n

TypeError: __init__() got an unexpected keyword argument 'gradient_boosting__n_estimators'